In [1]:
using Pkg; Pkg.activate("/home/roscar/work/cfgrib-project/cfgrib.jl")

Activating environment at `~/work/cfgrib-project/cfgrib.jl/Project.toml`


In [2]:
using cfgrib

┌ Info: Precompiling cfgrib [cb67cb4b-e5c8-45d4-aff0-3ae3657ca610]
└ @ Base loading.jl:1273


In [3]:
path = "/home/roscar/work/cfgrib-project/cfgrib.jl/tests/sample-data/era5-levels-members.grib"

"/home/roscar/work/cfgrib-project/cfgrib.jl/tests/sample-data/era5-levels-members.grib"

In [4]:
using DataStructures

In [5]:
using GRIB

## Types

In [13]:
struct OnDiskArray
    grib_path::String
    size::Tuple # TODO: rename -> shape
    offsets::OrderedDict
    missing_value::Any
    geo_ndim::Int
    dtype::Type
end

Base.size(a::OnDiskArray) = a.size # TODO: rename -> shape

In [14]:
struct Variable
    dimensions::Tuple{Vararg{String}}
    data::Union{Array, OnDiskArray}
    attributes::Dict{String, Any}
end

In [15]:
Variable(("1", ), [1,2], Dict("A"=>"b")) == Variable(("1", ), [1,2], Dict("A"=>"b"))

false

## build_dataset_components

In [16]:
a = "original GRIB coordinate for key:" *
     "arstarst"
println(a)

original GRIB coordinate for key:arstarst


In [17]:
#      Args
index = cfgrib.FileIndex(path, cfgrib.ALL_KEYS)
read_keys = []
kwargs = Dict()
#      Default
errors = "warn"
encode_cf = ("parameter", "time", "geography", "vertical")
squeeze = true
log = missing
LOG=log
# read_keys
time_dims = ("time", "step");

In [18]:
typeof(index)

cfgrib.FileIndex

In [19]:
collect(values(index.header_values))

59-element Array{Array,1}:
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 Missing[missing]                                
 [0]                                             
 [120]                                           
 [61]                                            
 ⋮                                               
 Missing[missing]                                
 SubString{String}["z", "t"]                     
 [0]                                             
 SubString{String}["instant"]                    
 [1]                                             
 [0]                   

In [20]:
fieldnames(cfgrib.FileIndex)

(:allowed_protocol_version, :grib_path, :index_path, :index_keys, :offsets, :message_lengths, :header_values, :filter_by_keys)

In [21]:
dimensions = OrderedDict()
variables = OrderedDict()
filter_by_keys = isdefined(index, :filter_by_keys) ? index.filter_by_keys : Dict();

In [29]:
#  THIS SHOULD BE A FOR LOOP
# param_id = index["paramId"][1]
param_id = index["paramId"][2]

130

In [30]:
var_index = cfgrib.filter(index, paramId=param_id);

## build_variable_components

https://github.com/ecmwf/cfgrib/blob/2f50a1ec0331be5ed7476aa4a0bf66fb2e388652/cfgrib/dataset.py#L433

### enforce_unique_attributes

In [31]:
function enforce_unique_attributes(index::cfgrib.FileIndex, attribute_keys::Array, filter_by_keys::Dict)
    attributes = OrderedDict()
    for key in attribute_keys
        values = index[key]

        if length(values) > 1
            throw("Attributes are not unique for $key: $(values)")
        end

        value = values[1]

        if !ismissing(value)
            attributes["GRIB_" * key] = value
        end
    end

    return attributes
end


enforce_unique_attributes (generic function with 1 method)

### encode_cf_first

In [32]:
function encode_cf_first(data_var_attrs::OrderedDict, encode_cf::Tuple{Vararg{String}}=("parameter", "time"), time_dims::Tuple{Vararg{String}}=("time", "step"))
    coords_map = cfgrib.ENSEMBLE_KEYS
    param_id = get(data_var_attrs, "GRIB_paramId", missing)
    data_var_attrs["long_name"] = "original GRIB paramId: $(param_id)"
    data_var_attrs["units"] = "1"

    if "parameter" in encode_cf
        if haskey(data_var_attrs, "GRIB_paramId")
            data_var_attrs["standard_name"] = data_var_attrs["GRIB_cfName"]
        end

        if haskey(data_var_attrs, "GRIB_name")
            data_var_attrs["long_name"] = data_var_attrs["GRIB_name"]
        end

        if haskey(data_var_attrs, "GRIB_units")
            data_var_attrs["units"] = data_var_attrs["GRIB_units"]
        end
    end
    
    if "time" in encode_cf
        if issubset(time_dims, cfgrib.ALL_REF_TIME_KEYS)
            append!(coords_map, time_dims)
        else
            throw("time_dims $(time_dims) is not a subset of $(cfgrib.ALL_REF_TIME_KEYS)")
        end
    else
        append!(coords_map, cfgrib.DATA_TIME_KEYS)
    end
    
    append!(coords_map, cfgrib.VERTICAL_KEYS)
    append!(coords_map, cfgrib.SPECTRA_KEYS)
    
    return coords_map
end

encode_cf_first (generic function with 3 methods)

In [37]:
function build_geography_coordinates(
        index, encode_cf, errors, log=LOG
    )
    
    first_message = cfgrib.first(var_index)
    geo_coord_vars = OrderedDict()
    grid_type = cfgrib.getone(index, "gridType")
    
    if "geography" in encode_cf && grid_type in cfgrib.GRID_TYPES_DIMENSION_COORDS
        geo_dims = ("latitude", "longitude")
        geo_shape = (cfgrib.getone(index, "Ny"), cfgrib.getone(index, "Nx"))
        latitudes = first_message["distinctLatitudes"]
        geo_coord_vars["latitude"] = Variable(
            ("latitude",), latitudes, cfgrib.COORD_ATTRS["latitude"]
        )
        
        if latitudes[1] > latitudes[end]
            geo_coord_vars["latitude"].attributes["stored_direction"] = "decreasing"
        end

        geo_coord_vars["longitude"] = Variable(
            ("longitude",),
            first_message["distinctLongitudes"],
            cfgrib.COORD_ATTRS["longitude"],
        )
    elseif "geography" in encode_cf && grid_type in cfgrib.GRID_TYPES_2D_NON_DIMENSION_COORDS
        throw("unimplemented")
    else
        throw("unimplemented")
    end
    
    return geo_dims, geo_shape, geo_coord_vars
end

build_geography_coordinates (generic function with 2 methods)

In [38]:
build_geography_coordinates(var_index, encode_cf, errors, log)

(("latitude", "longitude"), (61, 120), OrderedDict{Any,Any}("latitude" => Variable(("latitude",), [90.0, 87.0, 84.0, 81.0, 78.0, 75.0, 72.0, 69.0, 66.0, 63.0  …  -63.0, -66.0, -69.0, -72.0, -75.0, -78.0, -81.0, -84.0, -87.0, -90.0], Dict{String,Any}("units" => "degrees_north","stored_direction" => "decreasing","long_name" => "latitude","standard_name" => "latitude")),"longitude" => Variable(("longitude",), [0.0, 3.0, 6.0, 9.0, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0  …  330.0, 333.0, 336.0, 339.0, 342.0, 345.0, 348.0, 351.0, 354.0, 357.0], Dict{String,Any}("units" => "degrees_east","long_name" => "longitude","standard_name" => "longitude"))))

In [36]:
function build_valid_time(time::Array{Int, 1}, step::Number)
    step_s = step * 3600

    if length(size(time)) == 0 && length(size(step)) == 0
        data = time .+ step_s
        dims = Tuple{String}(("", ))
    elseif length(size(time)) > 0 && length(size(step)) == 0
        data = time .+ step_s
        dims = ("time", )
    elseif length(size(time)) == 0 && length(size(step)) > 0
        data = time .+ step_s
        dims = ("step", )
    else
        data = time .+ step_s
        dims = ("time", "step")
    end

    return dims, data
end

build_valid_time (generic function with 1 method)

In [ ]:
function build_valid_time(time::Array{Int, 1}, step::Array)
    @assert length(step) == 1
    return build_valid_time(time, step[1])
end

In [30]:
build_valid_time([1483228800, 1483272000, 1483315200, 1483358400], 0.0)

(("time",), [1.4832288e9, 1.483272e9, 1.4833152e9, 1.4833584e9])

In [184]:
function build_variable_components(index, encode_cf=(), filter_by_keys=Dict(), log=LOG, errors="warn", squeeze=true, read_keys=[], time_dims=("time", "step"))
    data_var_attrs_keys = cfgrib.DATA_ATTRIBUTES_KEYS
    data_var_attrs_keys = [data_var_attrs_keys; get(cfgrib.GRID_TYPE_MAP, index["gridType"][1], [])]
    data_var_attrs_keys = [data_var_attrs_keys; read_keys]

    data_var_attrs = enforce_unique_attributes(var_index, data_var_attrs_keys, filter_by_keys)

    coords_map = encode_cf_first(data_var_attrs, encode_cf, time_dims)

    coord_name_key_map = Dict()
    coord_vars = OrderedDict()

    for coord_key in coords_map
        values = index[coord_key]
        if length(values) == 1 && ismissing(values[1])
            #  TODO: Add logging
            #  @warn "Missing from GRIB Stream $(coord_key)"
            continue
        end
        
        coord_name = coord_key

        if ("vertical" in encode_cf && coord_key == "level" && haskey(data_var_attrs, "GRIB_typeOfLevel"))
            coord_name = data_var_attrs["GRIB_typeOfLevel"]
            coord_name_key_map[coord_name] = coord_key
        end
        
        attributes = Dict(
            "long_name" => "original GRIB coordinate for key: $(coord_key)($(coord_name))",
            "units"     => "1",
        )
        
        merge!(attributes, copy(get(cfgrib.COORD_ATTRS, coord_name, Dict())))
        
        data = sort(values, rev=get(attributes, "stored_direction", "none") == "decreasing")
        dimensions = (coord_name, )
        
        if squeeze && length(values) == 1
            data = data[1]
            typeof(data) == Array ? nothing : data = [data]
            dimensions = ()
        end
        
        coord_vars[coord_name] = Variable(dimensions, data, attributes)
    end
    
    header_dimensions = Tuple(d for (d, c) in pairs(coord_vars) if !squeeze || length(c.data) > 1)
    #  Loses information on which shape belongs to which dimension
    #  doesn't seem to matter though
    header_shape = Iterators.flatten(
        Tuple(size(coord_vars[d].data) for d in header_dimensions)
    )
    
    geo_dims, geo_shape, geo_coord_vars = build_geography_coordinates(index, encode_cf, errors)
    
    dimensions = (header_dimensions..., geo_dims...)
    shape = (header_shape..., geo_shape...)
    
    merge!(coord_vars, geo_coord_vars)
    
    offsets = OrderedDict{NTuple{length(header_dimensions), Int64}, Int}()
    for (header_values, offset) in var_index.offsets
        header_indexes = Array{Int}(undef, length(header_dimensions))
        for (i, dim) in enumerate(header_dimensions)
            coord_name = get(coord_name_key_map, dim, dim)
            coord_idx = findfirst(var_index.index_keys .== coord_name)
            header_value = header_values[coord_idx]
            header_indexes[i] = findfirst(coord_vars[dim].data .== header_value)
        end
        
        offsets[Tuple(header_indexes)] = offset
    end
    
    data = OnDiskArray(
        path,
        shape,
        offsets,
        missing,
        length(geo_dims),
        Float32
    )
    
    if haskey(coord_vars, "time") && haskey(coord_vars, "step")
        # add the 'valid_time' secondary coordinate
        dims, time_data = build_valid_time(
            coord_vars["time"].data,
            coord_vars["step"].data
        )
        attrs = cfgrib.COORD_ATTRS["valid_time"]
        coord_vars["valid_time"] = Variable(dims, time_data, attrs)
    end
    
    data_var_attrs["coordinates"] = join(keys(_d), " ")
    data_var = Variable3(dimensions, data, data_var_attrs)
    dims = OrderedDict((d => s) for (d, s) in zip(dimensions, size(data_var.data)))

    return dims, data_var, coord_vars
end

build_variable_components (generic function with 8 methods)

In [185]:
_d = build_variable_components(var_index, encode_cf, filter_by_keys, log, errors, squeeze, read_keys, time_dims)

(OrderedDict{Any,Any}("number" => 10,"time" => 4,"isobaricInhPa" => 2,"latitude" => 61,"longitude" => 120), Variable3(("number", "time", "isobaricInhPa", "latitude", "longitude"), OnDiskArray("/home/roscar/work/cfgrib-project/cfgrib.jl/tests/sample-data/era5-levels-members.grib", (10, 4, 2, 61, 120), OrderedDict((1, 1, 2) => 0,(2, 1, 2) => 14760,(3, 1, 2) => 29520,(4, 1, 2) => 44280,(5, 1, 2) => 59040,(6, 1, 2) => 73800,(7, 1, 2) => 88560,(8, 1, 2) => 103320,(9, 1, 2) => 118080,(10, 1, 2) => 132840…), missing, 2, Float32), Dict{String,Any}("GRIB_units" => "m**2 s**-2","long_name" => "Geopotential","GRIB_dataType" => "an","GRIB_totalNumber" => 10,"GRIB_jScansPositively" => 0,"GRIB_name" => "Geopotential","GRIB_gridType" => "regular_ll","GRIB_Ny" => 61,"GRIB_longitudeOfLastGridPointInDegrees" => 357.0,"GRIB_stepUnits" => 1…)), OrderedDict{Any,Any}("number" => Variable(("number",), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], Dict("units" => "1","long_name" => "ensemble member numerical id","standard_